# 2020: Week 6

In [1]:
# Setup
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [18]:
# Load data
rates = pd.read_excel(".\\inputs\\PD 2020 Wk 6 Input.xlsx", "GBP to USD conversion rate")
sales = pd.read_excel(".\\inputs\\PD 2020 Wk 6 Input.xlsx", "Sales")

## Explore 

In [19]:
print("Rates")
print("shape:", rates.shape)
print(rates.head())
print()
print("Sales")
print("shape:", sales.shape)
print(sales.head())

Rates
shape: (153, 2)
        Date British Pound to US Dollar
0 2020-01-31         1 GBP = 1.3205 USD
1 2020-01-30         1 GBP = 1.3086 USD
2 2020-01-29         1 GBP = 1.3021 USD
3 2020-01-28         1 GBP = 1.3023 USD
4 2020-01-27         1 GBP = 1.3058 USD

Sales
shape: (31, 4)
   Week  Year  Sales Value  US Stock sold (%)
0    27  2019        53025                 42
1    28  2019        49994                 44
2    29  2019        55236                 29
3    30  2019        76013                 33
4    31  2019        25544                 40


## Clean

In [20]:
#create week column
rates["week"] = rates["Date"].dt.week

#create year column
rates["year"] = rates["Date"].dt.year
rates.loc[(rates["Date"].dt.month == 12) & (rates["week"] == 1),"year"] = rates["year"] + 1  #account for crossover weeks
rates.loc[(rates["Date"].dt.month == 1) & (rates["week"] == 52),"year"] = rates["year"] - 1 

#extract conversion rates
s = []
for i in range(len(rates)):
    m = re.search(r"(?<== ).*?(?= USD)", rates["British Pound to US Dollar"][i])
    if m:
        r = float(m.group(0))
    else:
        r = np.nan
    s.append(r)  
rates["rate"] = s

#exlude Sat (5) and Sun (6)
rates = rates.loc[(rates["Date"].dt.weekday != 6) & (rates["Date"].dt.weekday != 5)]

#get min and max rates by week
weekly_rates = rates.groupby(["year","week"]).agg( {"rate" :["min", "max"]})
weekly_rates.columns = weekly_rates.columns.droplevel(0)
weekly_rates = weekly_rates.reset_index()

# create UK and US sales columns
sales["US Sales Value (GBP)"] = sales["US Stock sold (%)"] / 100 * sales["Sales Value"]
sales["UK Sales Value (GBP)"] = sales["Sales Value"] - sales["US Sales Value (GBP)"]

In [21]:
# join tables
df = sales.merge(weekly_rates, left_on=['Year', "Week"], right_on=['year', "week"])

# get min and max US sales
df["US Sales (USD) Best Case"] = df["US Sales Value (GBP)"] * df["max"]
df["US Sales (USD) Worst Case"] = df["US Sales Value (GBP)"] * df["min"]
df["US Sales Potential Variance"] = df["US Sales (USD) Best Case"] - df["US Sales (USD) Worst Case"]

# select columns to export
df = df[["Week", "Year","UK Sales Value (GBP)", "US Sales (USD) Best Case","US Sales (USD) Worst Case", "US Sales Potential Variance"]]

#create week column
df["week"] = "wk " + df["Week"].astype(str) + " " + df["Year"].astype(str)

#drop year and week, reorder
df = df[["week","UK Sales Value (GBP)", "US Sales (USD) Best Case","US Sales (USD) Worst Case", "US Sales Potential Variance" ]]

In [22]:
df.head()

,week,UK Sales Value (GBP),US Sales (USD) Best Case,US Sales (USD) Worst Case,US Sales Potential Variance
0,wk 36 2019,86802.66,62847.330352,61501.475776,1345.854576
1,wk 37 2019,115585.36,19709.930820,19429.373628,280.557192
2,wk 38 2019,37249.65,32413.635270,32172.901515,240.733755
3,wk 39 2019,70397.46,51655.868276,50812.439660,843.428616
4,wk 40 2019,51559.56,28575.653184,28464.463872,111.189312


## Export

In [23]:
df.to_csv(".\\outputs\\2020-06_data-preppin-output.csv", index=False)